# Food Vision With SegFormer and Hugging Face

# Helper Functions

## Visualization Helpers

In [46]:
def visualize_image_with_mask(image, mask=None, alpha=0.5, ignore_index=255):
    """
    Visualize an image, its segmentation mask, and overlay.

    Args:
        image (np.ndarray or PIL.Image.Image): RGB image.
        mask (np.ndarray or PIL.Image.Image): Segmentation mask (H x W).
        alpha (float): Transparency for overlay blending.
        ignore_index (int): Mask value to ignore (e.g., 255).
    """
    # --- Convert image to NumPy ---
    img = np.array(image)
    if img.ndim == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    if img.dtype != np.uint8:
        img = np.clip(img * 255, 0, 255).astype(np.uint8)
    h, w = img.shape[:2]

    mask_np = None
    if mask is not None:
        mask_np = np.array(mask, dtype=np.int64)

        # --- Resize mask to match image if needed ---
        if mask_np.shape[:2] != (h, w):
            mask_np = cv2.resize(mask_np, (w, h), interpolation=cv2.INTER_NEAREST)

        # --- Deterministic color map (tab20 colormap) ---
        cmap = plt.get_cmap("tab20", np.max(mask_np) + 1)
        color_mask = np.zeros_like(img, dtype=np.uint8)

        for cls_id in np.unique(mask_np):
            if cls_id == ignore_index:
                color = (128, 128, 128)  # gray for ignore
            elif cls_id == 0:
                color = (30, 30, 30)  # dark gray for background
            else:
                color = (np.array(cmap(cls_id)[:3]) * 255).astype(np.uint8)
            color_mask[mask_np == cls_id] = color

        # --- Blend overlay ---
        blended = cv2.addWeighted(img, 1 - alpha, color_mask, alpha, 0)
    else:
        blended = img

    # --- Plot ---
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.imshow(img)
    plt.title("🖼️ Original Image")
    plt.axis("off")

    plt.subplot(1, 3, 2)
    if mask_np is not None:
        plt.imshow(mask_np, cmap="tab20", vmin=0, vmax=np.max(mask_np))
        plt.title("🎭 Segmentation Mask")
    else:
        plt.text(0.5, 0.5, "No mask", ha="center", va="center", fontsize=12)
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.imshow(blended)
    plt.title("✨ Overlay (Image + Mask)")
    plt.axis("off")

    plt.tight_layout()
    plt.show()

    # --- Print metadata ---
    if mask_np is not None:
        valid = mask_np != ignore_index
        coverage = np.count_nonzero(valid) / mask_np.size * 100
        print(f"✅ Mask coverage: {coverage:.2f}% of image area")
        print(f"🟢 Unique classes: {np.unique(mask_np[valid]).tolist()}")
        print(f"📏 Image size: {img.shape}")
        print(f"🎭 Mask size: {mask_np.shape}")

In [47]:
import torch

def unnormalize_image(
    tensor: torch.Tensor,
    mean=(0.485, 0.456, 0.406),
    std=(0.229, 0.224, 0.225)
):
    """
    Reverse ImageNet normalization to make images display correctly.
    Args:
        tensor (torch.Tensor): Normalized image tensor (C, H, W)
    Returns:
        torch.Tensor: Unnormalized tensor in [0, 1]
    """
    tensor = tensor.detach().cpu()
    if tensor.ndim != 3 or tensor.shape[0] != 3:
        raise ValueError(f"Expected shape (3,H,W), got {tensor.shape}")
    mean = torch.tensor(mean).view(3, 1, 1)
    std = torch.tensor(std).view(3, 1, 1)
    tensor = tensor * std + mean
    return torch.clamp(tensor, 0, 1)

In [48]:
def plot_segmentation(images, masks, preds, n=4, save_path=None, unnormalize=True):
    """
    Visualize input images, ground truth masks, and model predictions.
    Args:
        images: (B,C,H,W)
        masks:  (B,H,W)
        preds:  (B,H,W)
    """
    images, masks, preds = images.cpu(), masks.cpu(), preds.cpu()
    plt.figure(figsize=(12, n * 3))
    cmap = plt.get_cmap("tab20")

    for i in range(min(n, images.size(0))):
        img = unnormalize_image(images[i]) if unnormalize else torch.clamp(images[i], 0, 1)
        img = img.permute(1, 2, 0).numpy()

        # --- Original ---
        plt.subplot(n, 3, i * 3 + 1)
        plt.imshow(img)
        plt.title("Original")
        plt.axis("off")

        # --- Ground Truth ---
        plt.subplot(n, 3, i * 3 + 2)
        plt.imshow(masks[i], cmap=cmap, vmin=0, vmax=torch.max(masks))
        plt.title("Ground Truth")
        plt.axis("off")

        # --- Prediction ---
        plt.subplot(n, 3, i * 3 + 3)
        plt.imshow(preds[i], cmap=cmap, vmin=0, vmax=torch.max(preds))
        plt.title("Prediction")
        plt.axis("off")

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, bbox_inches="tight", dpi=150)
        print(f"🖼️ Saved segmentation visualization to: {save_path}")
        plt.close()
    else:
        plt.show()

In [49]:
def plot_training_curves(results, save_path=None):
    """Visualize training metrics (loss, acc, mIoU) over epochs."""
    epochs = range(1, len(results["train_loss"]) + 1)

    plt.figure(figsize=(16, 5))

    # ---- Loss ----
    plt.subplot(1, 3, 1)
    plt.plot(epochs, results["train_loss"], "b-", marker="o", label="Train")
    plt.plot(epochs, results["test_loss"], "r-", marker="o", label="Val")
    plt.title("Loss per Epoch")
    plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.grid(True); plt.legend()

    # ---- Accuracy ----
    plt.subplot(1, 3, 2)
    plt.plot(epochs, results["train_acc"], "b-", marker="o", label="Train")
    plt.plot(epochs, results["test_acc"], "r-", marker="o", label="Val")
    plt.title("Pixel Accuracy per Epoch")
    plt.xlabel("Epoch"); plt.ylabel("Accuracy"); plt.grid(True); plt.legend()

    # ---- mIoU ----
    plt.subplot(1, 3, 3)
    plt.plot(epochs, results["train_iou"], "b-", marker="o", label="Train")
    plt.plot(epochs, results["test_iou"], "r-", marker="o", label="Val")
    plt.title("Mean IoU per Epoch")
    plt.xlabel("Epoch"); plt.ylabel("mIoU"); plt.grid(True); plt.legend()

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, bbox_inches="tight", dpi=150)
        print(f"✅ Saved training curves to {save_path}")
        plt.close()
    else:
        plt.show()

In [50]:
def display_random_sample(dataset: torch.utils.data.Dataset, seed: int = 42):
    """
    Displays a random image and its mask from a PyTorch dataset.

    Args:
        dataset (Dataset): A PyTorch dataset that returns (image, mask) pairs.
    """
    # Pick random index
    idx = random.randint(0, len(dataset) - 1)
    image, mask = dataset[idx]

    # If tensors → convert to NumPy (for plotting)
    if isinstance(image, torch.Tensor):
        image_np = image.permute(1, 2, 0).numpy()  # CHW → HWC
        image_np = np.clip(image_np, 0, 1)
    else:
        image_np = np.array(image)

    if isinstance(mask, torch.Tensor):
        mask_np = mask.numpy()
    else:
        mask_np = np.array(mask)

    # --- Print metadata ---
    print(f"🖼️ Sample Index: {idx}")
    # --- Visualize using helper ---
    visualize_image_with_mask(image_np, mask_np)

## Metric Helpers

In [51]:
def pixel_accuracy(preds, labels, ignore_index: int = 255):
    """Compute per-pixel accuracy (ignoring 'ignore_index' pixels)."""
    valid_mask = labels != ignore_index
    correct = (preds[valid_mask] == labels[valid_mask]).sum().item()
    total = valid_mask.sum().item()
    return correct / total if total > 0 else 0.0

In [52]:
def intersection_over_union(preds, labels, num_classes: int, ignore_index: int = 255):
    """Compute mean Intersection-over-Union (mIoU), ignoring 'ignore_index' pixels."""
    preds = preds.detach().cpu()
    labels = labels.detach().cpu()

    # Mask out ignored pixels
    mask = labels != ignore_index
    preds = preds[mask]
    labels = labels[mask]

    ious = []
    for cls in range(num_classes):
        pred_inds = preds == cls
        label_inds = labels == cls
        intersection = (pred_inds & label_inds).sum().item()
        union = (pred_inds | label_inds).sum().item()
        if union == 0:
            continue
        ious.append(intersection / union)

    return np.mean(ious) if ious else 0.0

## Checkpoint Utilities

In [53]:
def save_checkpoint(model, optimizer=None, scheduler=None, epoch=0, best_miou=0.0, path="ckpts", filename=None):
    """
    Save model weights + optimizer + scheduler state (for training resume).
    """
    os.makedirs(path, exist_ok=True)
    filename = filename or f"segformer_finetuned_epoch_{epoch}.ckpt"
    checkpoint_path = os.path.join(path, filename)

    torch.save({
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict() if optimizer else None,
        "scheduler_state_dict": scheduler.state_dict() if scheduler else None,
        "best_miou": best_miou,
    }, checkpoint_path)

    print(f"✅ Checkpoint saved: {checkpoint_path} (Best mIoU: {best_miou:.4f})")

def save_huggingface_export(model, processor, export_dir="./ckpts/segformer_finetuned"):
    """
    Save model + processor in Hugging Face format (for inference and sharing).
    """
    os.makedirs(export_dir, exist_ok=True)
    model.save_pretrained(export_dir)
    processor.save_pretrained(export_dir)
    print(f"🎯 Exported Hugging Face model and processor to: {export_dir}")

In [54]:
from transformers import SegformerForSemanticSegmentation, SegformerImageProcessor

def load_checkpoint(model, path, weights_only: bool = False, optimizer=None, scheduler=None, device="cuda"):
    """
    Load checkpoint (for resuming training).
    """
    checkpoint = torch.load(path, map_location=device, weights_only=weights_only)
    model.load_state_dict(checkpoint["model_state_dict"])
    if optimizer and checkpoint.get("optimizer_state_dict"):
        optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    if scheduler and checkpoint.get("scheduler_state_dict"):
        scheduler.load_state_dict(checkpoint["scheduler_state_dict"])
    epoch = checkpoint.get("epoch", 0)
    best_miou = checkpoint.get("best_miou", 0.0)
    print(f"✅ Loaded checkpoint from epoch {epoch+1} | Best mIoU={best_miou:.4f}")
    return checkpoint

def load_huggingface_export(model_dir="./ckpts/segformer_finetuned", device="cuda"):
    """
    Load model and processor saved via Hugging Face API (for inference).
    """
    model = SegformerForSemanticSegmentation.from_pretrained(model_dir)
    processor = SegformerImageProcessor.from_pretrained(model_dir)
    model.to(device)
    model.eval()
    print(f"🚀 Loaded Hugging Face model and processor from: {model_dir}")
    return model, processor

## Setup

In [55]:
import os
import random
import time
from typing import Tuple, Optional, Dict, List
from enum import Enum
import json
# Essentials
from tqdm import tqdm
import cv2
import numpy as np
import matplotlib.pyplot as plt
# Pytorch
import torch
import torchvision
# Hugging Face
from datasets import load_dataset

## Install required libraries

In [56]:
from packaging import version

# For this notebook to run with updated APIs, we need torch 1.12+ and torchvision 0.13+
required_torch = "1.12.0"
required_torchvision = "0.13.0"

if version.parse(torch.__version__) < version.parse(required_torch) or \
   version.parse(torchvision.__version__) < version.parse(required_torchvision):
    print("[INFO] torch/torchvision versions not as required, installing latest versions.")
    # You can change cu121 to cu118 or cpu as needed
    import os
    !pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
    # Reload torch and torchvision to reflect new versions
    import importlib
    importlib.reload(torch)
    importlib.reload(torchvision)

print(f"✅ torch version: {torch.__version__}")
print(f"✅ torchvision version: {torchvision.__version__}")

✅ torch version: 2.8.0+cu126
✅ torchvision version: 0.23.0+cu126


## Setup Platform

## Mount Drive

Uncomment the lines to mount google drive if using **google colab**.

In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Setup Device-Agnostic Code

In [58]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [59]:
print(f"✅ Using Device: {DEVICE}")

✅ Using Device: cuda


## Setup Seeds

In [60]:
def set_seeds(seed: int = 42):
    """Sets seed for reproducibility."""
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    random.seed(seed)

In [61]:
set_seeds()

# Data Preparation

## Dataset Structure

In [97]:
import torch
import numpy as np
from torch.utils.data import Dataset
from typing import Optional, Tuple, Literal
from enum import Enum
from transformers import BaseImageProcessor
from torchvision import transforms
from datasets import load_dataset
from PIL import Image

class FoodSeg103Dataset(Dataset):
    """
    Hugging Face + torchvision hybrid dataset wrapper for FoodSeg103.
    Supports:
        - HF processor (SegFormer-style)
        - torchvision transforms (augmentations)
        - both together, with configurable order
    """

    def __init__(
        self,
        hf_dataset,
        processor = None,
        transform: Optional[transforms.Compose] = None,
        transform_order: Literal["before", "after"] = "before",
    ):
        """
        Args:
            hf_dataset: Hugging Face dataset split (e.g., load_dataset("foodseg103", split="train"))
            processor: Hugging Face image processor (handles resizing, normalization, etc.)
            transform: torchvision transforms (for augmentation)
            transform_order: "before" or "after" processor application
        """
        self.dataset = hf_dataset
        self.processor = processor
        self.transform = transform
        self.transform_order = transform_order

        if self.processor is None and self.transform is None:
            print("⚠️ Warning: No processor or transform provided.")

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
      item = self.dataset[idx]
      image = item["image"]   # PIL Image
      label = item["label"]   # PIL mask

      # ======================================================
      # CASE 1 — Processor only
      # ======================================================
      if self.processor is not None and self.transform is None:
          encoded = self.processor(image, segmentation_maps=label, return_tensors="pt")
          return encoded["pixel_values"].squeeze(0), encoded["labels"].squeeze(0).long()

      # ======================================================
      # CASE 2 — Transform only
      # ======================================================
      if self.processor is None and self.transform is not None:
          image = self.transform(image)
          if isinstance(image, Image.Image):  # if transform didn’t convert to tensor
              image = transforms.ToTensor()(image)
          label = torch.as_tensor(np.array(label, dtype=np.int64), dtype=torch.long)
          return image, label

      # ======================================================
      # CASE 3 — Processor + Transform hybrid
      # ======================================================
      if self.processor is not None and self.transform is not None:
          if self.transform_order == "before":
              # Apply torchvision transforms first, then processor
              image = self.transform(image)
              if isinstance(image, torch.Tensor):
                  image = transforms.ToPILImage()(image)
              encoded = self.processor(image, segmentation_maps=label, return_tensors="pt")
              pixel_values = encoded["pixel_values"].squeeze(0)
              labels = encoded["labels"].squeeze(0).long()
              return pixel_values, labels
          else:
              # Apply processor first, then torchvision transforms
              encoded = self.processor(image, segmentation_maps=label, return_tensors="pt")
              image = encoded["pixel_values"].squeeze(0)
              labels = encoded["labels"].squeeze(0).long()

              # Convert back to PIL for torchvision transforms
              image = transforms.ToPILImage()(image)
              image = self.transform(image)
              # Ensure tensor output
              if isinstance(image, Image.Image):
                  image = transforms.ToTensor()(image)
              return image, labels

      # ======================================================
      # CASE Fallback
      # ======================================================
      image = transforms.ToTensor()(image)
      label = torch.as_tensor(np.array(label, dtype=np.int64), dtype=torch.long)
      return image, label

    def __repr__(self):
        transform_str = str(self.transform) if self.transform else "None"
        processor_str = self.processor.__class__.__name__ if self.processor else "None"
        split_name = getattr(self.dataset, "split", "unknown")
        return (
            f"Dataset: FoodSeg103\n"
            f"  Split: {split_name}\n"
            f"  Samples: {len(self)}\n"
            f"  Processor: {processor_str}\n"
            f"  Transform: {transform_str}\n"
            f"  Order: {self.transform_order}\n"
        )


## Class Mappings

In [63]:
def load_class_mappings(json_path: str) -> Tuple[Optional[Dict[int, str]], Optional[Dict[int, str]]]:
    """
    Load id2label and label2id mappings from a JSON file.

    Args:
        json_path (str): Path to the JSON file like:
            {
              "0": "background",
              "1": "candy",
              "2": "egg tart",
              ...
            }

    Returns:
        tuple: (id2label: dict[int, str], label2id: dict[str, int])
    """
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # Convert string keys to integers for id2label
    id2label = {int(k): v for k, v in data.items()}
    label2id = {v: int(k) for k, v in id2label.items()}

    return id2label, label2id

In [64]:
id2label, label2id = load_class_mappings("/content/drive/MyDrive/datasets/foodSeg103/classnames.json")

print("First 5 id2label:")
print(dict(list(id2label.items())[:5]))

print("\nFirst 5 label2id:")
print(dict(list(label2id.items())[:5]))

First 5 id2label:
{0: 'background', 1: 'candy', 2: 'egg tart', 3: 'french fries', 4: 'chocolate'}

First 5 label2id:
{'background': 0, 'candy': 1, 'egg tart': 2, 'french fries': 3, 'chocolate': 4}


## Create DataLoaders

In [88]:
class Split(Enum):
    TRAIN = "train"
    VALIDATION = "validation"
    TEST = "test"
    ALL = "all"


def create_dataloaders_from_hf(
    dataset_name: str,
    split: Split = Split.ALL,
    processor = None,
    train_transform: Optional[transforms.Compose] = None,
    val_transform: Optional[transforms.Compose] = None,
    test_transform: Optional[transforms.Compose] = None,
    transform_order: Literal["before", "after"] = "before",
    batch_size: int = 4,
    num_workers: int = 2,
    device: str = None,
) -> Tuple[
    Optional[torch.utils.data.DataLoader],
    Optional[torch.utils.data.DataLoader],
    Optional[torch.utils.data.DataLoader],
]:
    """Creates PyTorch DataLoaders with per-split transforms + processor hybrid support."""

    def _build_loader(split_name: str, transform:Optional[transforms.Compose] = None, is_train: bool = False):
        ds = load_dataset(dataset_name, split=split_name)
        dataset = FoodSeg103Dataset(
            ds,
            processor=processor,
            transform=transform,
            transform_order=transform_order,
        )
        _device = device if device else ("cuda" if torch.cuda.is_available() else "cpu")
        pin_mem = True if _device == "cuda" else False

        return torch.utils.data.DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=is_train,
            drop_last=is_train,
            num_workers=num_workers,
            pin_memory=pin_mem,
        )

    train_loader = val_loader = test_loader = None

    if split in (Split.TRAIN, Split.ALL):
        try:
          train_loader = _build_loader("train", transform=train_transform, is_train=True)
        except ValueError:
            print(f"⚠️ No 'train' split found in dataset: {dataset_name}")
    if split in (Split.VALIDATION, Split.ALL):
        try:
          val_loader = _build_loader("validation", transform=val_transform)
        except ValueError:
            print(f"⚠️ No 'validation' split found in dataset: {dataset_name}")
    if split in (Split.TEST, Split.ALL):
        try:
            test_loader = _build_loader("test", transform=test_transform)
        except ValueError:
            print(f"⚠️ No 'test' split found in dataset: {dataset_name}")

    return train_loader, val_loader, test_loader

# Model Pipeline

## SegFormer Model

In [87]:
from transformers import SegformerForSemanticSegmentation, SegformerConfig, SegformerImageProcessor, AutoImageProcessor

class Segformer:
    """
    A flexible builder for creating and configuring SegFormer models
    from the Hugging Face Transformers library.
    """

    def __init__(
        self,
        model_name: str = "nvidia/segformer-b0-finetuned-ade-512-512",
        num_classes: int = 104,
        ignore_mismatched_sizes: bool = True,
        dropout: float = 0.1,
        device: str = "cuda",
        processor = None,
        id2label: Optional[Dict[int, str]] = None,
        label2id: Optional[Dict[str, int]] = None
    ):
        """
        Initialize the SegformerBuilder.

        Args:
            model_name (str): Pretrained SegFormer model name or path.
            num_classes (int): Number of segmentation classes.
            ignore_mismatched_sizes (bool): Allow different output head sizes.
            use_pretrained (bool): Whether to load pretrained weights.
            device (str): Device to load model on ("cuda" or "cpu").
        """
        self.model_name = model_name
        self.num_classes = num_classes
        self.ignore_mismatched_sizes = ignore_mismatched_sizes
        self.device = device if device else ("cuda" if torch.cuda.is_available() else "cpu")
        self.id2label = id2label
        self.label2id = label2id

        # Load model config Parameters
        self.config = SegformerConfig.from_pretrained(
            model_name,
            num_labels=num_classes,
            hidden_dropout_prob=dropout,
            id2label=self.id2label,
            label2id=self.label2id
        )

        # Load model weights
        self.model = SegformerForSemanticSegmentation.from_pretrained(
            model_name,
            config=self.config,
            ignore_mismatched_sizes=ignore_mismatched_sizes
        )
        # Load image processor (for transforms)
        self.processor = processor or AutoImageProcessor.from_pretrained(model_name)

        # Set model to device
        self.model.to(self.device)

    def get_model(self) -> torch.nn.Module:
        """Return the PyTorch model."""
        return self.model

    def get_processor(self):
        """Return the Hugging Face image processor for transforms."""
        return self.processor

    def freeze_encoder(self, freeze=True):
        """Freeze or unfreeze the encoder layers."""
        for param in self.model.segformer.encoder.parameters():
            param.requires_grad = not freeze
        print(f"Encoder frozen: {freeze}")

    def freeze_decoder(self, freeze=True):
        """Freeze or unfreeze the decoder layers."""
        for param in self.model.decode_head.parameters():
            param.requires_grad = not freeze
        print(f"Decoder frozen: {freeze}")

    def unfreeze_all(self):
        """Unfreeze all parameters."""
        for param in self.model.parameters():
            param.requires_grad = True
        print("All model parameters are trainable.")

    def summary(self):
        """Print basic info about the model."""
        print("\n🧩 SegFormer Model Summary\n")
        print(f"Model: {self.model_name}")
        print(f"Classes: {self.num_classes}")
        print(f"Device: {self.device}")
        print(f"Parameters: {sum(p.numel() for p in self.model.parameters()):,}")
        print(f"Trainable: {sum(p.numel() for p in self.model.parameters() if p.requires_grad):,}")
        # Add shape info
        size = self.processor.size.get("height", 512)
        dummy = torch.randn(1, 3, size, size).to(self.device)
        with torch.no_grad():
            out = self.model(pixel_values=dummy)
        print(f"Output keys: {list(out.keys())}")
        print(f"Logits shape: {out.logits.shape}")

## Trainning Engine

In [67]:
from torch.amp import autocast, GradScaler

def train_step(model, dataloader, scaler, optimizer, loss_fn, num_classes: int, device: str):
    """Run one training epoch."""
    model.train()
    train_loss, train_acc, train_iou = 0, 0, 0
    autocast_device = "cuda" if device == "cuda" else "cpu"

    for images, labels in tqdm(dataloader, leave=False, desc="Training"):
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)

        try:
            # ---- Forward + Backward with AMP ----
            with autocast(device_type=autocast_device, enabled=(device == "cuda")):
                outputs = model(pixel_values=images)
                logits = torch.nn.functional.interpolate(
                    outputs.logits,
                    size=labels.shape[-2:],
                    mode="bilinear",
                    align_corners=False
                )
                loss = loss_fn(logits, labels)

            # ---- Gradient Scaling ----
            if scaler:
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                loss.backward()
                optimizer.step()

            # ---- Metrics ----
            train_loss += loss.item()
            preds = torch.argmax(logits, dim=1)
            train_acc += pixel_accuracy(preds, labels)
            train_iou += intersection_over_union(preds, labels, num_classes)

            del loss, outputs, logits
            torch.cuda.empty_cache()

        except RuntimeError as e:
            if "out of memory" in str(e).lower():
                print("⚠️ Skipping batch due to OOM error.")
                torch.cuda.empty_cache()
                continue
            else:
                raise e

    n = len(dataloader)
    return train_loss / n, train_acc / n, train_iou / n


def eval_step(model, dataloader, loss_fn, num_classes: int, device: str):
    """Evaluate model for one epoch."""
    model.eval()
    test_loss, test_acc, test_iou = 0, 0, 0
    autocast_device = "cuda" if device == "cuda" else "cpu"

    with torch.inference_mode():
        for images, labels in tqdm(dataloader, leave=False, desc="Evaluating"):
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            with autocast(device_type=autocast_device, enabled=(device == "cuda")):
                outputs = model(pixel_values=images)
                logits = torch.nn.functional.interpolate(
                    outputs.logits,
                    size=labels.shape[-2:],
                    mode="bilinear",
                    align_corners=False
                )
                loss = loss_fn(logits, labels)
                preds = torch.argmax(logits, dim=1)

            test_loss += loss.item()
            test_acc += pixel_accuracy(preds, labels)
            test_iou += intersection_over_union(preds, labels, num_classes)

            del loss, outputs, logits
            torch.cuda.empty_cache()

    n = len(dataloader)
    return test_loss / n, test_acc / n, test_iou / n


In [71]:
from __future__ import annotations
from typing import Optional, Callable, Union, Tuple, Dict, Literal
from pathlib import Path
import os, time, torch, numpy as np
from torch.amp import GradScaler, autocast
from tqdm import tqdm

# Type aliases
DataLoader = torch.utils.data.DataLoader
Tensor = torch.Tensor
MetricTriplet = Tuple[float, float, float]
ResumeMode = Literal["auto", "latest", "best", "fresh"]
Checkpoint = Dict[str, object]


def train(
    model: torch.nn.Module,
    train_dataloader: DataLoader,
    val_dataloader: DataLoader,
    optimizer: torch.optim.Optimizer,
    loss_fn: Callable[[Tensor, Tensor], Tensor],
    *,
    epochs: int,
    num_classes: int,
    device: Union[str, torch.device] = "cuda",
    scheduler: Optional[torch.optim.lr_scheduler._LRScheduler] = None,
    save_dir: Union[str, Path] = "ckpts",
    vis_dir: Optional[Union[str, Path]] = "visualizations",
    auto_save: bool = True,
    vis_every: int = 5,
    min_best_miou: float = 0.0,
    max_keep_checkpoints: int = 3,
    last_ckpt_path: Optional[Union[str, Path]] = None,
    best_ckpt_path: Optional[Union[str, Path]] = None,
    resume_mode: ResumeMode = "auto",  # "auto" | "latest" | "best" | "fresh"
    find_latest_ckpt: Optional[Callable[[Union[str, Path]], Optional[Path]]] = None,
    load_checkpoint: Callable[..., Checkpoint] = None,
    save_checkpoint: Callable[..., None] = None,
    train_step: Callable[..., MetricTriplet] = None,
    eval_step: Callable[..., MetricTriplet] = None,
    plot_segmentation: Optional[Callable[..., None]] = None,
    plot_training_curves: Optional[Callable[..., None]] = None,
    seed: Optional[int] = None,
) -> Tuple[Checkpoint, float, int]:
    """
    Train loop for SegFormer with AMP, checkpointing, and auto-resume.

    Returns
    -------
    ckpt : dict
        Last-loaded checkpoint (empty if starting fresh)
    best_miou : float
        Best validation mIoU achieved
    start_epoch : int
        Epoch to resume from
    """

    # ============================================================
    # 🔹 Setup
    # ============================================================
    os.makedirs(save_dir, exist_ok=True)
    device = torch.device(device)
    model.to(device)
    scaler = GradScaler() if device.type == "cuda" else None
    if seed is not None:
        torch.manual_seed(seed)
        np.random.seed(seed)

    # ============================================================
    # 🔹 Helper: Find latest checkpoint
    # ============================================================
    def _default_find_latest_ckpt(path: str | Path) -> Optional[Path]:
        ckpts = [f for f in os.listdir(path) if f.startswith("last_checkpoint_") and f.endswith(".ckpt")]
        if not ckpts:
            return None
        ckpts.sort(key=lambda x: int(x.split("_")[-1].split(".")[0]))
        return Path(path) / ckpts[-1]

    find_latest_ckpt = find_latest_ckpt or _default_find_latest_ckpt

    # ============================================================
    # 🔹 Checkpoint Resume Logic
    # ============================================================
    latest = Path(last_ckpt_path) if last_ckpt_path else find_latest_ckpt(save_dir)
    best = Path(best_ckpt_path) if best_ckpt_path else Path(save_dir) / "best_segformer.ckpt"

    match resume_mode:
        case "latest" if latest:
            ckpt_path, tag = latest, "latest"
        case "best" if best.exists():
            ckpt_path, tag = best, "best"
        case "auto" if latest:
            ckpt_path, tag = latest, "latest"
        case "auto" if best.exists():
            ckpt_path, tag = best, "best"
        case _:
            ckpt_path, tag = None, "fresh"

    if ckpt_path and load_checkpoint:
        print(f"🔁 Resuming from {tag} checkpoint: {ckpt_path}")
        ckpt = load_checkpoint(model, ckpt_path, False, optimizer, scheduler, device)
        best_miou, start_epoch = ckpt.get("best_miou", 0.0), ckpt.get("epoch", 0) + 1
    else:
        print("🆕 Starting fresh training...")
        ckpt, best_miou, start_epoch = {}, 0.0, 0

    # ============================================================
    # 🔹 Initialize Metrics
    # ============================================================
    results = {k: [] for k in ["train_loss", "train_acc", "train_iou", "val_loss", "val_acc", "val_iou", "epoch_time"]}

    # ============================================================
    # 🔹 Training Loop
    # ============================================================
    for epoch in range(start_epoch, epochs):
        start_t = time.perf_counter()
        print(f"\n🌱 Epoch [{epoch + 1}/{epochs}]")

        train_loss, train_acc, train_iou = train_step(model, train_dataloader, scaler, optimizer, loss_fn, num_classes, device)
        val_loss, val_acc, val_iou = eval_step(model, val_dataloader, loss_fn, num_classes, device)

        if scheduler:
            scheduler.step()

        epoch_time = time.perf_counter() - start_t
        tqdm.write(f"🕒 {epoch_time:.2f}s | "
                   f"Train: L={train_loss:.4f}, A={train_acc:.4f}, mIoU={train_iou:.4f} | "
                   f"Val: L={val_loss:.4f}, A={val_acc:.4f}, mIoU={val_iou:.4f}")

        # Record metrics
        for k, v in zip(results.keys(), [train_loss, train_acc, train_iou, val_loss, val_acc, val_iou, epoch_time]):
            results[k].append(v)

        # ========================================================
        # 🔹 Save Checkpoints
        # ========================================================
        if auto_save:
            if val_iou > best_miou and val_iou >= min_best_miou:
                best_miou = val_iou
                save_checkpoint(model, optimizer, scheduler, epoch, best_miou, path=save_dir, filename="best_segformer.ckpt")
                print(f"🏆 New best model saved (mIoU={best_miou:.4f})")

            ckpt_file = Path(save_dir) / f"last_checkpoint_{epoch + 1}.ckpt"
            save_checkpoint(model, optimizer, scheduler, epoch, best_miou, path=save_dir, filename=ckpt_file.name)

            # Rotate checkpoints
            ckpts = sorted([f for f in os.listdir(save_dir) if f.startswith("last_checkpoint_")],
                           key=lambda x: int(x.split("_")[-1].split(".")[0]))
            if len(ckpts) > max_keep_checkpoints:
                old = Path(save_dir) / ckpts[0]
                os.remove(old)
                print(f"🧹 Removed old checkpoint: {old}")

        # ========================================================
        # 🔹 Visualization
        # ========================================================
        if (epoch + 1) % vis_every == 0 and plot_segmentation:
            model.eval()
            imgs, labels = next(iter(val_dataloader))
            imgs, labels = imgs.to(device), labels.to(device)
            with torch.inference_mode(), autocast(device_type=device.type if device.type == "cuda" else "cpu"):
                outputs = model(pixel_values=imgs)
                logits = torch.nn.functional.interpolate(outputs.logits, size=labels.shape[-2:], mode="bilinear", align_corners=False)
                preds = torch.argmax(logits, dim=1)
            plot_segmentation(imgs, labels, preds, n=min(4, len(imgs)),
                              save_path=Path(vis_dir) / f"vis_epoch_{epoch + 1}.png")
            if plot_training_curves:
                plot_training_curves(results, save_path=Path(vis_dir) / "training_curves.png")

    # ============================================================
    # 🔹 Summary
    # ============================================================
    print(f"\n🎯 Training complete! 🏆 Best mIoU={best_miou:.4f} | ⏱️ Avg Epoch={np.mean(results['epoch_time']):.2f}s")
    return ckpt, best_miou, start_epoch

## Fine Tunning

In [98]:
# EPOCHS = 30
# LR = 1e-4
# WEIGHT_DECAY = 1e-4
# T_MAX = 20
# IGNORE_INDEX = 255
# CHECKPOINT_DIR = "drive/MyDrive/ckpts"
# NUM_WORKERS = os.cpu_count() or 2 # Use 2 Workers as default
# BATCH_SIZE = 4
# NUM_CLASSES = 104
# DROP_RATE = 0.1

# # Get Class Mappings
# id2label,label2id = load_class_mappings("drive/MyDrive/datasets/foodSeg103/classnames.json")

# # Initialize model and builder
# builder = Segformer(num_classes=NUM_CLASSES, device=DEVICE, dropout=DROP_RATE, id2label=id2label, label2id=label2id)
# model = builder.get_model()
# # Get Image Processor from builder
# processor = builder.get_processor()

# # Initialize Dataloaders
# train_loader, validation_loader, _ = create_dataloaders_from_hf(dataset_name="EduardoPacheco/FoodSeg103", batch_size=BATCH_SIZE, processor=processor)

# # Loss (ignore_index=255 if present in dataset)
# loss_fn = torch.nn.CrossEntropyLoss(ignore_index=IGNORE_INDEX)

# # Optimizer and scheduler
# optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=T_MAX)

# # Display Summary
# print(builder.summary())

# # Fine-tune
# if __name__ == "__main__":
#     results = train(
#     model=model,
#     train_dataloader=train_loader,
#     test_dataloader=validation_loader,
#     optimizer=optimizer,
#     loss_fn=loss_fn,
#     epochs=EPOCHS,
#     device=DEVICE,
#     num_classes=NUM_CLASSES,
#     scheduler=scheduler,
#     save_dir=CHECKPOINT_DIR,
#     vis_every=10,  # visualize predictions every 10 epochs
#     auto_save=True
# )



In [ ]:
from torchvision import transforms

EPOCHS = 50
LR = 1e-4
WEIGHT_DECAY = 1e-4
T_MAX = EPOCHS
IGNORE_INDEX = 255
CHECKPOINT_DIR = "drive/MyDrive/ckpts"
NUM_WORKERS = os.cpu_count() or 2 # Use 2 Workers as default
BATCH_SIZE = 4
NUM_CLASSES = 104
DROP_RATE = 0.1

# Get Class Mappings
id2label,label2id = load_class_mappings("drive/MyDrive/datasets/foodSeg103/classnames.json")

# Initialize builder and processor
builder = Segformer(num_classes=NUM_CLASSES, device=DEVICE, dropout=DROP_RATE, id2label=id2label, label2id=label2id)
processor = builder.get_processor()
print(f"Processor: {processor.__class__.__name__}")
# Initialize model
model = builder.get_model()

# ============================================================
# 🔹 Data Augmentation (Boost Generalization)
# ============================================================
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(512, scale=(0.8, 1.0)),     # scale jitter
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),
])
val_transform = None  # Keep validation clean

# Initialize Dataloaders
train_loader, validation_loader, _ = create_dataloaders_from_hf(
    dataset_name="EduardoPacheco/FoodSeg103",
    batch_size=BATCH_SIZE,
    processor=processor, # Applied Image Processor
    train_transform=train_transform,
    val_transform=val_transform,
    transform_order="before",  # Apply torchvision → then processor
)

# Loss (ignore_index=255 if present in dataset)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=IGNORE_INDEX)

# Optimizer and scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=T_MAX)

# Load Checkpoint
ckpt_path = "drive/MyDrive/ckpts/best_segformer.ckpt"
checkpoint = torch.load(ckpt_path, map_location=DEVICE, weights_only=False)

model.load_state_dict(checkpoint["model_state_dict"])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
if "scheduler_state_dict" in checkpoint:
    scheduler.load_state_dict(checkpoint["scheduler_state_dict"])

start_epoch = EPOCHS - checkpoint["epoch"] + 1
best_miou = checkpoint["best_miou"]

print(f"✅ Loaded best checkpoint from epoch {start_epoch-1} with mIoU={best_miou:.4f}")

# 🔹 Optional: lower LR to refine learning
for g in optimizer.param_groups:
    g["lr"] = 5e-5  # reduce LR by half

# 🔓 Unfreeze encoder for deeper fine-tuning
builder.unfreeze_all()

print(builder.summary())
print("Current LR:", optimizer.param_groups[0]["lr"])

# Fine-tune
if __name__ == "__main__":
    results = train(
    model=model,
    train_dataloader=train_loader,
    val_dataloader=validation_loader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    epochs=EPOCHS,
    device=DEVICE,
    num_classes=NUM_CLASSES,
    scheduler=scheduler,
    save_dir=CHECKPOINT_DIR,
    vis_every=5,  # visualize predictions every 5 epochs
    vis_dir="drive/MyDrive/ckpts/visualizations",
    auto_save=True,
    train_step=train_step, # Add train_step argument
    eval_step=eval_step, # Add eval_step argument
    best_ckpt_path= ckpt_path,
    resume_mode= "best",
    load_checkpoint=load_checkpoint,
    save_checkpoint=save_checkpoint,
)

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b0-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([104]) in the model instantiated
- decode_head.classifier.weight: found shape torch.Size([150, 256, 1, 1]) in the checkpoint and torch.Size([104, 256, 1, 1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/transformers/image_processing_base.py:417: UserWarning: The following named arguments are not valid for `SegformerImageProcessor.__init__` and were ignored: 'reduce_labels'
  image_processor = cls(**image_processor_dict)


Processor: SegformerImageProcessor
⚠️ No 'test' split found in dataset: EduardoPacheco/FoodSeg103
✅ Loaded best checkpoint from epoch 43 with mIoU=0.2656
All model parameters are trainable.

🧩 SegFormer Model Summary

Model: nvidia/segformer-b0-finetuned-ade-512-512
Classes: 104
Device: cuda
Parameters: 3,740,872
Trainable: 3,740,872
Output keys: ['logits']
Logits shape: torch.Size([1, 104, 128, 128])
None
Current LR: 5e-05
🔁 Resuming from best checkpoint: drive/MyDrive/ckpts/best_segformer.ckpt
✅ Loaded checkpoint from epoch 8 | Best mIoU=0.2656

🌱 Epoch [9/50]


🕒 3193.31s | Train: L=1.7286, A=0.5901, mIoU=0.1245 | Val: L=1.2157, A=0.6809, mIoU=0.2426
✅ Checkpoint saved: drive/MyDrive/ckpts/last_checkpoint_9.ckpt (Best mIoU: 0.2656)

🌱 Epoch [10/50]


Training:  21%|██▏       | 266/1245 [07:30<23:44,  1.45s/it]

## Export Checkpoint for Inference

In [ ]:
# Save Hugging Face compatible export
save_huggingface_export(model, processor, export_dir="/drive/MyDrive/models/segformer_finetuned")

# Model Analytics



## Load Saved Model

In [ ]:
load_huggingface_export(model, path="/drive/MyDrive/models/segformer_best.ckpt", device=builder.device)
model.eval()

## Plot Curves

In [ ]:
plot_loss_curves(training_results, save_path="models/checkpoints/training_plots.png")

## Plot Training Time

In [ ]:
def plot_training_time(results, save_path=None):
    """Plot time taken per epoch and display total + average."""
    epoch_times = results.get("epoch_time", [])
    if not epoch_times:
        print("[WARN] No epoch_time data found in results.")
        return

    epochs = range(1, len(epoch_times) + 1)
    avg_time = np.mean(epoch_times)
    total_time = np.sum(epoch_times)

    plt.figure(figsize=(8, 5))
    plt.plot(epochs, epoch_times, marker='o', color='mediumseagreen', linewidth=2)
    plt.title("⏱️ Training Time per Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Time (seconds)")
    plt.grid(alpha=0.3)
    plt.xticks(epochs)
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, bbox_inches="tight")
        print(f"✅ Saved training time plot to {save_path}")

    plt.show()
    print(f"📊 Total Training Time: {total_time:.2f} seconds")
    print(f"⚙️  Average Epoch Time: {avg_time:.2f} seconds")

In [ ]:
plot_training_time(results, save_path="models/checkpoints/training_time.png")

## Visualize Predictions

In [ ]:
import torch
import matplotlib.pyplot as plt
from torchvision.utils import make_grid

def visualize_predictions(model, dataloader, device, id2label, num_samples=4):
    """Visualize a few segmentation predictions with class colors."""
    model.eval()
    model.to(device)

    images, labels = next(iter(dataloader))
    images, labels = images.to(device), labels.to(device)

    with torch.no_grad():
        outputs = model(pixel_values=images)
        logits = torch.nn.functional.interpolate(
            outputs.logits, size=labels.shape[-2:], mode="bilinear", align_corners=False
        )
        preds = torch.argmax(logits, dim=1).cpu()

    # Display a few samples
    for i in range(min(num_samples, len(images))):
        img = images[i].permute(1, 2, 0).cpu().numpy()
        pred_mask = preds[i].numpy()
        true_mask = labels[i].cpu().numpy()

        fig, ax = plt.subplots(1, 3, figsize=(12, 4))
        ax[0].imshow(img)
        ax[0].set_title("Original Image")

        ax[1].imshow(true_mask, cmap="tab20")
        ax[1].set_title("Ground Truth")

        ax[2].imshow(pred_mask, cmap="tab20")
        ax[2].set_title("Predicted Mask")

        for a in ax: a.axis("off")
        plt.tight_layout()
        plt.show()


In [ ]:
visualize_predictions(model, validation_loader, builder.device, builder.id2label, num_samples=4)